In [6]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

## Design of Experiment  
This Notebook is used to calculate the amount of Silica and DOPC in the sample. The inputs are the a Silica to DOPC surface areas, and the outputs are arrays with the mass of each material to use to obtain the desired surface area. 

In [7]:
silica_concentration = 2 #mg/mL
sample_volume = 10 #mL
bilayer_silica_ratio_lst = [0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

In [8]:
################### Material Properties ###################
silica_molar_mass = 60.08 #g/mol
silica_density = 2.65 #g/cm3
silica_particle_diameter = 26 #nm

DOPC_molar_mass = 785 #g/mol 
DOPC_SA_molecule = 0.6 #nm2/molecule

In [9]:
def calculate_total_silica_NP_SA(silica_mass):
    silica_moles = silica_mass/silica_molar_mass/1000
    silica_atoms = silica_moles*6.02e23
    silica_NP_volume = 4/3*(silica_particle_diameter/2)**3*np.pi #nm3
    silica_atoms_volume = silica_mass/(silica_density*1e-18) #nm3
    silica_NP_quantitity = silica_atoms_volume/silica_NP_volume
    silica_NP_SA = 4*np.pi*(silica_particle_diameter/2)**2
    total_silica_NP_SA = silica_NP_SA*silica_NP_quantitity #nm2 
    return total_silica_NP_SA

def calculate_total_DOPC_SA(DOPC_mass):
    DOPC_moles = DOPC_mass/DOPC_molar_mass/1000
    DOPC_molecules = DOPC_moles*6.02e23
    total_DOPC_SA = DOPC_molecules*DOPC_SA_molecule
    total_DOPC_SA = total_DOPC_SA/2
    return total_DOPC_SA

def DOPC_SA_to_Mass(DOPC_SA):
    DOPC_SA = DOPC_SA*2
    DOPC_molecules = DOPC_SA/DOPC_SA_molecule
    DOPC_moles = DOPC_molecules/6.02e23
    DOPC_mass = DOPC_moles*DOPC_molar_mass*1000
    return DOPC_mass

def calculate_SA(silica_mass, bilayer_silica_ratio):
    silica_SA = calculate_total_silica_NP_SA(silica_mass)
    DOPC_SA = silica_SA
    DOPC_mass = DOPC_SA_to_Mass(DOPC_SA)*bilayer_silica_ratio
    return DOPC_mass


In [10]:
silica_mass = silica_concentration*sample_volume
DOPC_Mass_list = []
for j in range(len(bilayer_silica_ratio_lst)):
    bilayer_silica_ratio = bilayer_silica_ratio_lst[j]
    DOPC_mass = calculate_SA(silica_mass, bilayer_silica_ratio)
    DOPC_Mass_list.append(DOPC_mass)
DOPC_mass_array = np.array(DOPC_Mass_list).reshape(-1,1)

In [11]:
silica_mass_array = np.array([silica_mass]*len(bilayer_silica_ratio_lst)).reshape(-1,1)
final_mass_array = np.hstack((silica_mass_array, DOPC_mass_array))

## Mass Array

This array contains the mass of silica and DOPC to create each sample

In [12]:
pd.DataFrame(np.round(final_mass_array, 2), columns = ['Silica Mass (mg)', 'DOPC Mass (mg)'])

,Silica Mass (mg),DOPC Mass (mg)
0,20.0,3.03
1,20.0,4.54
2,20.0,6.06
3,20.0,7.57
4,20.0,9.08
5,20.0,10.60


## Concentration Array

We can convert the mass array to a concentration array in mg/mL. 

In [13]:
sample_volume = 10 #mL 
conc_array = final_mass_array/sample_volume

In [14]:
pd.DataFrame(np.round(conc_array, 2), columns = ['Silica Conc (mg/mL)', 'DOPC Conc (mg/mL)'])

,Silica Conc (mg/mL),DOPC Conc (mg/mL)
0,2.0,0.30
1,2.0,0.45
2,2.0,0.61
3,2.0,0.76
4,2.0,0.91
5,2.0,1.06


## Volume Array

By knowing the concentration of the stock solutions used, we can calculate the volume of each stock solution to add to the sample. 

In [15]:
stock_solution_silica = 60 #mg/mL
stock_solution_DOPC = 50 #mg/mL

In [16]:
volume_array = final_mass_array.copy()
volume_array[:,0] = final_mass_array[:,0]/stock_solution_silica*1000
volume_array[:,1] = final_mass_array[:,1]/stock_solution_DOPC*1000

In [17]:
volume_array_f = np.hstack((volume_array, np.array(bilayer_silica_ratio_lst).reshape(-1,1)))
df = pd.DataFrame(np.round(np.round(volume_array, 2)), columns = ['Silica Stock (60 mg/mL) (uL)', 'DOPC Stock (50 mg/mL) (uL)'])
df

,Silica Stock (60 mg/mL) (uL),DOPC Stock (50 mg/mL) (uL)
0,333.0,61.0
1,333.0,91.0
2,333.0,121.0
3,333.0,151.0
4,333.0,182.0
5,333.0,212.0


In [42]:
#df.to_excel('../../Volumes/250225/Silica_DOPC_ratio_10mL.xlsx')